In [ ]:
import mysql.connector
import mysql.connector.errorcode
import pandas as pd
import numpy as np

cnx = "";
unique_meters: pd.DataFrame = None;
def connect_to_database():
    global cnx
    cnx = mysql.connector.connect(
            user="guilhermec",
            password="6B&54V$Z7rkD",
            host="iot-laager.c3wepnzq2di2.us-east-1.rds.amazonaws.com",
            database="DLAAGERIOT"
    )


In [ ]:
import pandas as pd
import datetime

connect_to_database()
if cnx and cnx.is_connected():
    global unique_meters
    with cnx.cursor() as cursor:
        cursor.execute(
            """
                select bu.name, r.rgi, m.serial_number `meter_code`, m.seal_number, so.created_at,
                m.positive_count, r.status,
                concat('https://iot.laager.com.br/so/images/full/', soi.hash_value) from residences r 
                inner join meters m on (m.residence_id = r.id)
                inner join commercial_services cs on(r.commercial_service_id = cs.id)
                inner join business_units bu on (cs.business_unit_id = bu.id)
                inner join service_orders so on (so.residence_id = r.id)
                inner join service_order_images soi on (soi.service_order_id = so.id)
                where bu.id = 762 and r.status = 'ACTIVATED' and soi.stage = 'POS' and m.seal_number <> 0
            """)
        
        data = cursor.fetchall()
        meters = pd.DataFrame(data, columns=["business_unit", "rgi", "meter_serial_number", "device_serial_number",
                                             "created_at", "current_reading", "status", "photo_link"])
        meters["created_at"] = meters["created_at"].apply(lambda date: datetime.datetime.strftime(date, "%d/%m/%Y %H:%M:%S"));
        display(meters)
        meters.to_excel("data_with_all_devices.xlsx", index=False)


    cnx.close();
else:
    print("Não conectado")

In [ ]:
import pandas as pd
import datetime
data = pd.read_excel("./data.xlsx")
data["created_at"] = pd.to_datetime(data["created_at"]);
data["created_at"] = data["created_at"].apply(lambda date: datetime.datetime.strftime(date, "%d/%m/%Y %H:%M:%S"));
display(data)

In [ ]:
import requests
photo_links = data["photo_link"].to_numpy();
first_url = photo_links[0]


In [ ]:
import os
import requests

for unique_rgi in data["rgi"].unique():
    if unique_rgi in ("00227855-3", "01314032-9"):
        subdata = data[data["rgi"]==unique_rgi]
        dir_to_create = fr"./{unique_rgi}"
        os.mkdir(dir_to_create)
        for i, photo_link in enumerate(subdata["photo_link"].unique()):
            req = requests.request("GET", url=photo_link);
            with open(fr"{dir_to_create}/image{i}.jpg", "wb") as handler:
                handler.write(req.content);